In [2]:
# wget을 활용해서 bert 모델 다운로드 가능
#-*- coding: utf-8 -*-
import os
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

if "bert" not in os.listdir():
  os.makedirs("bert")
else:
  pass

import zipfile
import shutil
         
bert_zip = zipfile.ZipFile('multi_cased_L-12_H-768_A-12.zip')
bert_zip.extractall('bert')
 
bert_zip.close()

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

copytree("bert/multi_cased_L-12_H-768_A-12", "bert")

--2020-02-07 04:12:03--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 2404:6800:4008:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip.1’

multi_cased_L-12_H- 100%[===================>] 632.19M   101MB/s    in 6.7s    

2020-02-07 04:12:10 (94.4 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip.1’ saved [662903077/662903077]



In [3]:
import tensorflow as tf

import pandas as pd
import numpy as np  
import re
import pickle

import keras as keras
from keras.models import load_model
from keras import backend as K
from keras import Input, Model
from keras import optimizers

from keras import backend as K
from keras.layers import Layer

import codecs
from tqdm import tqdm
import shutil
import json

Using TensorFlow backend.


In [4]:
import os
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
import warnings
import tensorflow as tf
warnings.filterwarnings(action='ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
tf.logging.set_verbosity(tf.logging.ERROR)

In [6]:
!pip install keras-bert
!pip install keras-radam

In [0]:
from keras_bert import load_trained_model_from_checkpoint, load_vocabulary
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps

from keras_radam import RAdam

In [8]:
path = "/content/gdrive/My Drive/Colab Notebooks/inven_crawling_data"
os.listdir(path)

['kms_crawl.csv']

In [0]:
DATA_COLUMN = "title"
LABEL_COLUMN = "main_cate"

In [0]:
data = pd.read_csv(os.path.join(path, "kms_crawl.csv"), encoding='UTF-8')

In [11]:
data.head()

,Unnamed: 0,main_cate,sub_cate,title
0,0,도전!기자단,일기,초이트워커 [3]
1,1,도전!기자단,일기,초이트워커 [2]
2,2,도전!기자단,일기,초이트워커 [1.5] 코디편
3,3,도전!기자단,일기,초이트워커 [1]
4,4,도전!기자단,육성,악당오르카의 나이트워커 육성기 -3-


In [0]:
data = data.drop(['Unnamed: 0', 'sub_cate'],axis=1)

#### 불용어 정리, 너무 쓸모없는 데이터가 많다. 

In [0]:
from nltk.tokenize import RegexpTokenizer
retokenize = RegexpTokenizer("[\w]+")

In [0]:
text_list = []
for sentence in data[DATA_COLUMN].tolist():
  text_list.append(retokenize.tokenize(sentence))
  
for i in range(len(data[DATA_COLUMN])) :
  data[DATA_COLUMN][i] = " ".join(text_list[i])

In [15]:
data[0:11]

,main_cate,title
0,도전!기자단,초이트워커 3
1,도전!기자단,초이트워커 2
2,도전!기자단,초이트워커 1 5 코디편
3,도전!기자단,초이트워커 1
4,도전!기자단,악당오르카의 나이트워커 육성기 3
5,도전!기자단,건달 대적자의 하루1
6,도전!기자단,옛날 맵 태국 플로팅마켓 의 비밀
7,도전!기자단,2 사막에 위치한 왕국 아리안트 BGM
8,도전!기자단,추억의 옛날 전사 스킬과 설명
9,도전!기자단,1 장난감으로 이루어진 마을 루디브리엄 BG


In [16]:
import copy
category_names = copy.deepcopy(data[LABEL_COLUMN].unique())
# index를 통하여 나중에 네이밍을 다시 해줘야 하기 때문에 deepcopy를 합니다.

print(len(data[LABEL_COLUMN].unique()))
print(category_names[-1])

13
길드/친구/농장찾기


In [17]:
data[LABEL_COLUMN] = np.array(data[LABEL_COLUMN].factorize()[0])
data[LABEL_COLUMN].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

**sklearn 을 통하여 훈련,테스트 데이터 나누기.**

In [0]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.25, random_state=123)

In [19]:
print(len(train_data))

18618


In [0]:
SEQ_LEN = 50
BATCH_SIZE = 16
EPOCHS=2
LR=1e-5

pretrained_path ="bert"
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [0]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)

In [0]:
class inherit_Tokenizer(Tokenizer):
  def _tokenize(self, text):
        if not self._cased:
            text = text
            
            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [0]:
tokenizer = inherit_Tokenizer(token_dict)



*   항목 추가
*   항목 추가



In [24]:
tokenizer.tokenize("메이플 세계관 7 完 검은마법사의 소멸")

['[CLS]',
 '메',
 '##이',
 '##플',
 '세계',
 '##관',
 '7',
 '完',
 '검',
 '##은',
 '##마',
 '##법',
 '##사의',
 '소',
 '##멸',
 '[SEP]']

In [0]:
def convert_data(data_df):
    global tokenizer
    indices, targets = [], []
    for i in data.index:
        try :
          ids, segments = tokenizer.encode(data_df[DATA_COLUMN][i], max_len=SEQ_LEN)
          indices.append(ids)
          targets.append(data_df[LABEL_COLUMN][i])
        except :
          continue  
    items = list(zip(indices, targets))
    
    indices, targets = zip(*items)
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)], np.array(targets)

def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    
    
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x, data_y = convert_data(data_df)

    return data_x, data_y

In [0]:
train_x, train_y = load_data(train_data)
test_x, test_y = load_data(test_data)

In [0]:
layer_num = 12
model = load_trained_model_from_checkpoint( #bert.5h인가 불러오기
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

In [28]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 50)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 50)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 50, 768), (1 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 50, 768)      1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
def get_bert_finetuning_model(model):
  inputs = model.inputs[:2]
  dense = model.layers[-3].output

  
  outputs = keras.layers.Dense(len(data[LABEL_COLUMN].unique()), activation='softmax',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



  bert_model = keras.models.Model(inputs, outputs)
  bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])
  
  return bert_model

In [30]:
sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init = tf.variables_initializer([v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables])
sess.run(init)

bert_model = get_bert_finetuning_model(model)
history = bert_model.fit(train_x, train_y, epochs=2, batch_size=16, verbose = 1, validation_data=(test_x, test_y), shuffle=True)

Train on 18618 samples, validate on 6206 samples
Epoch 1/2
18618/18618 [==============================] - 272s 15ms/step - loss: 1.9716 - acc: 0.3269 - val_loss: 1.5472 - val_acc: 0.4700
Epoch 2/2
18618/18618 [==============================] - 253s 14ms/step - loss: 1.3834 - acc: 0.5331 - val_loss: 1.2609 - val_acc: 0.5893


In [0]:
path = "gdrive/My Drive/Colab Notebooks/"

In [0]:
bert_model.save_weights(path+"/inven_multi_classification.h5")

In [34]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 50)           0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 50)           0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 50, 768), (1 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 50, 768)      1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [0]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN)
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def category_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = bert_model.predict(data_x)

    category_index = np.argmax(predict)
    
    category =  category_names[category_index]
  
    return category

In [36]:
category_evaluation_predict("나만 이걸 보고 울었는지 모르겠지만 우리나라의 역사의 슬픔과 왕과 신하의 신뢰와 믿음... 그 모든것이 절 슬프게 하였네요... ")

'자유게시판'

In [37]:
category_evaluation_predict("10랩 전사 무기 팝니다 ")

'사고팔고'